In [ ]:
#Importing dependencies
import numpy as np
import string
import random
import re
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.models import Sequential
from numpy import array, argmax, random, take
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import RNN, SimpleRNN, LSTM,  Embedding, RepeatVector
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
%matplotlib inline
#For plotting the matplotlib graphs in notebook

In [ ]:
Data_path="D:\\Google Drive\\Training\\Book\\0.Chapters\\Chapter12 RNN and LSTM\\Datasets"

In [ ]:
import pandas as pd
column_names = ['word1', 'word2', 'word3']

input_3gram = pd.read_csv(Data_path+ "\\3Gram_love_data.txt", delimiter='\t', names=column_names) #Importing csv file with column names
print("shape of data", input_3gram.shape)

In [ ]:
print("Few sample records from data \n", input_3gram.sample(10))

In [ ]:
print("\nFrequency of word1 values \n", input_3gram["word1"].value_counts())
print("\nFrequency of word2 values \n", input_3gram["word2"].value_counts())

In [ ]:
"""
Finding our words to create dictionary
Here we find unique values in each column and save each of those values .
Later which we will take the unique value for the entire appened columns
This will be our vocabulary list,which are the unique words in our data file
"""
unique_words = []
for i in list(input_3gram.columns.values):
    for j in pd.unique(input_3gram[i]):
        unique_words.append(j)
unique_words = np.unique(unique_words)


print('Count of unique words overall:', len(unique_words))
print('unique words list:', unique_words)

In [ ]:
"""
creating our word:indice pair dictionary and inverse
Here will be creating two dictonary values
word_indices : This contains each words mapped to an unique digit 
indices_words : This contains each digits mapped to a word in the same sequence as word_indices 
"""
word_indices = dict((w, i) for i, w in enumerate(unique_words))
indices_words = dict((i, w) for i, w in enumerate(unique_words))

print("word_indices dictionary \n",word_indices)
print("word_indices.keys \n", word_indices.keys())
print("word_indices.values \n", word_indices.values())
print("\n ########################################\n")
print("indices_words dictionary \n", indices_words)
print("indices_words keys \n",indices_words.keys())
print("indices_words values \n",indices_words.values())

In [ ]:
### Onehot encoding of word1
word1 = input_3gram['word1'].map(word_indices)
word1_onehot = keras.utils.to_categorical(np.array(word1), num_classes=len(word_indices))
print("word1_onehot shape is ",word1_onehot.shape)

In [ ]:
#Lets take example of two different words
print("The word in row 0 is -->"+input_3gram['word1'][0])
print("The one hot encoded version of the word in row 0 is \n",word1_onehot[0])

print("\nThe word in row 500 is --> "+input_3gram['word1'][500])
print("The one hot encoded version of the word in row 500 is \n",word1_onehot[500])

In [ ]:
##one hot encoding for word2 and word3 
word2 = input_3gram['word2'].map(word_indices)
word2_onehot = keras.utils.to_categorical(np.array(word2), num_classes=len(word_indices))
print("word2_onehot shape is ",word2_onehot.shape)

word3 = input_3gram['word3'].map(word_indices)
word3_onehot = keras.utils.to_categorical(np.array(word3), num_classes=len(word_indices))
print("word3_onehot shape is ",word3_onehot.shape)

### First ANN Model

In [ ]:
ANN_model1 = Sequential()
ANN_model1.add(Dense(10, input_dim=word1_onehot.shape[1], activation='sigmoid'))
ANN_model1.add(Dense(word2_onehot.shape[1] ,activation='softmax'))
ANN_model1.summary()

In [ ]:
ANN_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = ANN_model1.fit(word1_onehot, word2_onehot, epochs=20, batch_size=50,  verbose=1)

In [ ]:
#We will see what the 1st hidden layer output representation of the data  
# to predict the hidden layer activations, 
# let's rewrite first layer of our model and give it the weights from fully trained previous model
model1_hidden = Sequential()
model1_hidden.add(Dense(10, input_dim=word1_onehot.shape[1], weights=ANN_model1.layers[0].get_weights()))
model1_hidden.add(Activation('sigmoid'))

In [ ]:
# Getting the hidden layer activations
model1_hidden_output = model1_hidden.predict(word1_onehot)
#peak into our hidden layer activations
print("The hidden layer output for every record - Shape of it \n", model1_hidden_output.shape)
print("Few five records from hidden layer \n",model1_hidden_output[:5])

In [ ]:
"""
We append the input words of the words2 column in the output of the h1 layer,this gives us the combined input representation
"""
word2_hidden_append = np.append(model1_hidden_output,word2_onehot, axis=1)
print("word2_hidden_append Shape", word2_hidden_append.shape)

In [ ]:
ANN_model2 = Sequential()
ANN_model2.add(Dense(10, input_dim=word2_hidden_append.shape[1], activation='sigmoid'))
ANN_model2.add(Dense(word3_onehot.shape[1], activation='softmax'))
ANN_model2.summary()

In [ ]:
ANN_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = ANN_model2.fit(word2_hidden_append, word3_onehot, epochs=20, batch_size=50,  verbose=1)

In [ ]:
# A predict function that takes input word1 and word2; and predict word3 
#1. take the input word , and represent them using digits from the word_indices dictonary values
#2. getting the intermediate hidden nodes for word1
#3. appending hidden activations with word2 as final test set
#4. prediction on this test set
def two_step_pred(words_in):

    index_input=word_indices[words_in[0]]
    indices_in = keras.utils.to_categorical(index_input, num_classes=len(word_indices))
    indices_in=indices_in.reshape(1,len(word_indices))
    h1_test = model1_hidden.predict(indices_in) # getting our intermediate hidden activations from model1h
    
    
    index_input2=word_indices[words_in[1]]
    indices_in2 = keras.utils.to_categorical(index_input2, num_classes=len(word_indices))
    indices_in2= indices_in2.reshape(1,len(word_indices))
    X2_test = np.append(h1_test, indices_in2, axis=1) #preparing final test data by appending hidden with word2
    
    yhat = ANN_model2.predict_classes(X2_test) #predicting final output from model2
    
    print("Input words --> ", words_in)
    print("Predicted word --> ", indices_words[yhat[0]])

In [ ]:
two_step_pred(['love', 'it'])
two_step_pred(['love', 'to'])
two_step_pred(['love', 'the'])

## RNNs

In [ ]:
model = Sequential()
model.add(SimpleRNN(4, use_bias=False, input_shape=(2,2)))
model.add(Dense(3, use_bias=False, activation='softmax'))
model.summary()

In [ ]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(2,2)))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(4,2)))
model.add(Dense(3, activation='softmax'))
model.summary()

## Word prediction using RNN model

In [ ]:
word1_word2 = input_3gram[['word1','word2']]
for i in list(word1_word2.columns.values):
    word1_word2[i] = word1_word2[i].map(word_indices)

word1_word2=np.array(word1_word2)
#The same data is reshaped with similar structure but appended with 1 value to make it 3d array
word1_word2=np.reshape(word1_word2,(word1_word2.shape[0],2,1))
word1_word2_onehot = keras.utils.to_categorical(np.array(word1_word2), num_classes=len(word_indices))
print("word1_word2_onehot shape", word1_word2_onehot.shape)

In [ ]:
print("time steps" , word1_word2_onehot.shape[1])
print("Input nodes" , word1_word2_onehot.shape[2])
print("output nodes" , word3_onehot.shape[1])

In [ ]:
model_rnn = Sequential()
#model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, input_data_dim)))
model_rnn.add(SimpleRNN(30, input_shape=(word1_word2_onehot.shape[1],word1_word2_onehot.shape[2]))) 
model_rnn.add(Dense(word3_onehot.shape[1], activation='softmax'))
model_rnn.summary()

In [ ]:
# compile network
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_rnn.fit(word1_word2_onehot, word3_onehot, epochs=20)

In [ ]:
def rnn_word_pred(in_text):
    print("Input is - " , in_text)
    encoded = [word_indices[i] for i in in_text]
    encoded = np.array(encoded).reshape(1,2,1)
    encoded =keras.utils.to_categorical(np.array(encoded), num_classes=len(word_indices))
    ypred = model_rnn.predict_classes(encoded, verbose=0)[0]
    print("Output is --> " ,indices_words[ypred])

In [ ]:
rnn_word_pred(['love', 'it'])
rnn_word_pred(['love', 'to'])
rnn_word_pred(['love', 'the'])

## RNN for Long Sequences

In [ ]:
longseq_3gram = open(Data_path+'\\Long_sequence_3gram.csv').read().lower()
print(longseq_3gram[495:801])
print(longseq_3gram[30615:31000])

In [ ]:
#Replace comma with space
longseq_3gram1= longseq_3gram.replace(',',' ').replace('\r','')
print(longseq_3gram1[495:750])
print(longseq_3gram1[30615:30800])

In [ ]:
#Unique characters in our dataset we then sort it
chars = sorted(list(set(longseq_3gram1)))
print("Unique Characters in the text \n ",chars)
#\n is character string for new line, we dont need that in our dictionary of chars
chars.remove('\n')
print("\n Character after removing newline symbol \'\\n\'",chars)
print("\n overall chars count", len(chars))

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
print("characters to indices dictionary\n", char_indices)
indices_char = dict((i, c) for i, c in enumerate(chars))
print("indices to char dictionary\n", indices_char)
print('unique chars: ', {len(chars)})

### Char to index on full data

In [ ]:
data = longseq_3gram1.splitlines()
##Adding a space at the end
data = [i+' ' for i in data]

##mapping our data into numbers
sentences = [[char_indices[j] for j in i] for i in data ]
print(data[0], sentences[0])
print(data[10], sentences[1])
print(data[20], sentences[2])
print(data[100], sentences[3])
print(data[400], sentences[400])
print(data[4000], sentences[4000])
print(data[9000], sentences[9000])
##Number of sentences
print("Number of sentences ", len(sentences))

### Converting the sentences to RNN friendly data which can be used to generate new sequences

**Take one sentence, iterate through it till the length of sentence is reached:**

* **Step 1** 0:0+14: X; and 14th position: y >> observation 1
* **Step 2** 1:14: X; and 15th position: y >> observation 2
* **Step 3** …we do this till the length of sentence is reached

Take next sentence and repeat the same


In [ ]:
#Since all the sentences may not be of same length,it is neccessary to make them consistent when passing to keras
#We select a sequence length
Seq_ln = 14
X = []
y = []
for i in sentences:
    for j in range(len(i)-Seq_ln):
        X.append(i[j:j+Seq_ln])
        y.append(i[j+Seq_ln])
len(X), len(y)

### Sample

In [ ]:
print("data[0:2]=", data[0:2])
print("sentences[0:2]=", sentences[0:2])

for i in range (0,20):
    print("X[",i,"]=", X[i],"y[",i,"]=", y[i])

In [ ]:
#The first row is the X's first row up to 14 character
#The second row is the X's first row starting from second character up to 14 character
#The third row is the X's first row starting from third character up to 14 character and so on 
X=np.array(X)
X1=np.reshape(X,(X.shape[0],X.shape[1],1))
X1=keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
print(X1.shape)

In [ ]:
#Target Variable
y[:10]
#Reshapig our label for model
y1 = np.array(y)
# one hot encode outputs
y1 = keras.utils.to_categorical(np.array(y), num_classes=len(char_indices))
y1.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.20)
print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape)
print("y_test.shape", y_test.shape)

In [ ]:
#building the model
model_RNN2 = Sequential()
##model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, data_dim)))
model_RNN2.add(SimpleRNN(16, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model_RNN2.add(Dense(len(char_indices)))
model_RNN2.add(Activation('softmax'))
model_RNN2.summary()

In [ ]:
# compile network
model_RNN2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_RNN2.fit(X_train, y_train, epochs=30, verbose=1, validation_data=(X_test, y_test))
model_RNN2.save_weights("char_rnn_model_weights_v1.hdf5")

In [ ]:
weightsfile_model_RNN2= Data_path+"\\Pre_trained_models\\char_rnn_model_weights_v1.hdf5"
model_RNN2.load_weights(weightsfile_model_RNN2)

# compile network
model_RNN2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_RNN2.fit(X_train, y_train, epochs=2, verbose=1)

In [ ]:
#function to prepare test input
def prepare_input(in_text):
    X1 = np.array([char_indices[i] for i in in_text]).reshape(1,14,1)
    X1=keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
    return(X1)
#function to loop our preditions
def complete_pred(in_text):
    #original_text = in_text
    #generated = in_text
    completion = ''
    while True:
        x = prepare_input(in_text)
        pred = model_RNN2.predict_classes(x, verbose=0)[0]

        next_char = indices_char[pred]

        in_text = in_text[1:] + next_char
        completion += next_char

        if len(completion)> 20 or next_char == ' ':
            return completion

In [ ]:
in_text = 'officials say '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted word ---> ", out_word)
in_text = 'how dangerous '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'political and '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'whatever they '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'of particular '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)

## LSTM Models

In [ ]:
#building the model
model_LSTM = Sequential()
#model1.add(LSTM('number of hidden nodes in each rnn cell', input_shape=(timesteps, data_dim)))
model_LSTM.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model_LSTM.add(Dense(len(char_indices)))
model_LSTM.add(Activation('softmax'))
model_LSTM.summary()

In [ ]:
# compile network
model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_LSTM.fit(X_train, y_train, epochs=30, verbose=1)
model_LSTM.save_weights("char_LSTM_model_weights_v1.hdf5")

In [ ]:
weightsfile_model_LSTM= Data_path+"\\Pre_trained_models\\char_LSTM_model_weights_v1.hdf5"
model_LSTM.load_weights( weightsfile_model_LSTM)

# compile network
model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_LSTM.fit(X_train, y_train,epochs=2, verbose=1)

In [ ]:
#function to prepare test input
def prepare_input1(in_text):
    X1 = np.array([char_indices[i] for i in in_text]).reshape(1,14,1)
    X1= keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
    return(X1)
#function to loop our preditions
def complete_pred1(in_text):
    #original_text = in_text
    #generated = in_text
    completion = ''
    while True:
        x = prepare_input1(in_text)
        pred = model_LSTM.predict_classes(x, verbose=0)[0]
        next_char = indices_char[pred]

        in_text = in_text[1:] + next_char
        completion += next_char

        if len(completion)> 20 or next_char == ' ':
            return completion

In [ ]:
in_text = 'the emergence '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'officials say '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'and sentenced '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'a combination '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'and according '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)

### Few more predictions and Comparions with Standard RNN

In [ ]:
in_text = 'how dangerous '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'political and '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'of particular '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'whatever they '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

## Case Study – Language Translation Project 

In [ ]:
raw_data= open(Data_path+"\\fra-eng\\fra.txt", mode='rt', encoding='utf-8').read()
raw_data=raw_data.strip().split('\n')
raw_data=[i.split('\t') for i in raw_data]
lang1_lang2_data=array(raw_data)
print(lang1_lang2_data)
print("Overall pairs", len(lang1_lang2_data))

In [ ]:
# Remove punctuation
lang1_lang2_data[:,0] = [word.translate(str.maketrans('', '', string.punctuation)) for word in lang1_lang2_data[:,0]]
lang1_lang2_data[:,1] = [word.translate(str.maketrans('', '', string.punctuation)) for word in lang1_lang2_data[:,1]]

print(lang1_lang2_data)

In [ ]:
## convert text to lowercase
for word in range(len(lang1_lang2_data)):
    lang1_lang2_data[word,0] = lang1_lang2_data[word,0].lower()
    lang1_lang2_data[word,1] = lang1_lang2_data[word,1].lower()
print(lang1_lang2_data)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lang1_lang2_data[:, 0])
lang1_tokens=tokenizer
lang1_vocab_size = len(lang1_tokens.word_index) + 1
print("lang1_vocab_size", lang1_vocab_size)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lang1_lang2_data[:, 1])
lang2_tokens=tokenizer
lang2_vocab_size = len(lang2_tokens.word_index) + 1
print("lang2_vocab_size", lang2_vocab_size)

In [ ]:
# split data into train and test set
train, test = train_test_split(lang1_lang2_data, test_size=0.1, random_state = 44)

In [ ]:
lang1_seq_length=15
lang2_seq_length=15

X_train_seq=lang1_tokens.texts_to_sequences(train[:, 0])
X_train= pad_sequences(X_train_seq,lang1_seq_length,padding='post')

Y_train_seq=lang2_tokens.texts_to_sequences(train[:, 1])
Y_train= pad_sequences(Y_train_seq,lang2_seq_length,padding='post')

X_test_seq=lang1_tokens.texts_to_sequences(test[:, 0])
X_test= pad_sequences(X_test_seq,lang1_seq_length,padding='post')

Y_test_seq=lang2_tokens.texts_to_sequences(test[:, 1])
Y_test= pad_sequences(Y_test_seq,lang2_seq_length,padding='post')

print("X_train.shape", X_train.shape)
print("Y_train.shape",Y_train.shape)
print("X_test.shape",X_test.shape)
print("Y_test.shape", Y_test.shape)

In [ ]:
print("Text data", [train[5, 0]])
print('Numbers sequence', X_train_seq[5])
print('Padded Sequence', X_train[5])

### Model Building

In [ ]:
model = Sequential()
model.add(Embedding(lang1_vocab_size, 256, input_length=lang1_seq_length, mask_zero=True))
model.add(LSTM(128))
model.add(RepeatVector(lang2_seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(lang2_vocab_size, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
history = model.fit(X_train, Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1),  epochs=1, verbose=1, batch_size=1024)
model.save_weights('Eng_fra_model.hdf5')

In [ ]:
model.load_weights(Data_path+"\\Pre_trained_models\\Eng_fra_model.hdf5")

### Prediction

In [ ]:
def one_line_prediction(text1):
    
    def to_lines(text):
          sents = text.strip().split('\n')
          sents = [i.split('\t') for i in sents]
          return sents
    small_input = to_lines(text1)
    small_input = array(small_input)
    
    # Remove punctuation
    small_input[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in small_input[:,0]]
    # convert text to lowercase
    for i in range(len(small_input)):
        small_input[i,0] = small_input[i,0].lower()

    #encode and pad sequences
    small_input_seq=lang1_tokens.texts_to_sequences(small_input[0])
    small_input= pad_sequences(small_input_seq,lang1_seq_length,padding='post')
   

    #Load the model
    #Eng French Model
    #model.load_weights('/content/drive/My Drive/Training/Book/0.Chapters/Chapter12 RNN and LSTM/1.Archives/Eng_fra_model_v2.hdf5')

    pred_seq = model.predict_classes(small_input[0:1].reshape((small_input[0:1].shape[0],small_input[0:1].shape[1])))
    
    def num_to_word(n, tokens):
          for word, index in tokens.word_index.items():
              if index == n:
                  return word
          return None

    Lang2_text = []
    for word_num in pred_seq:
          sing_pred = []
          for i in range(len(word_num)):
                t = num_to_word(word_num[i], lang2_tokens)
                if i > 0:
                    if (t == num_to_word(word_num[i-1], lang2_tokens)) or (t == None):
                        sing_pred.append('')
                    else:
                        sing_pred.append(t)
                else:
                      if(t == None):
                              sing_pred.append('')
                      else:
                              sing_pred.append(t) 
          Lang2_text.append(' '.join(sing_pred))
    return(Lang2_text)

In [ ]:
Input_sentences=["have a great Good day",
                 "Do you speak English",
                 "I do not know your language",
                 "I need help",
                 "Thank you very much",
                 "Where can I get this",
                 "How much does it cost",
                 "Where is the bathroom",
                 "Where is the ATM",
                 "I am a visitor here",
                 "Excuse me",
                 "What do you do for living",
                 "Here is my passport"]

for sent in Input_sentences:
  print([sent] , " -->",one_line_prediction(sent))